In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy
from scipy.stats import norm
from scipy.stats import uniform

In [107]:
#read data from "MacroSeries.txt"

macro_data = pd.read_csv("MacroSeries.txt", header = None)
macro_data.columns = ["c", "k", "w", "r", "y"]
c_vec = np.array(macro_data["c"])
k_vec = np.array(macro_data["k"])
w_vec = np.array(macro_data["w"])
r_vec = np.array(macro_data["r"])
y_vec = np.array(macro_data["y"])

In [108]:
def take_uniform_draws(S = 1000, T=100):
    m = uniform.rvs(0, 1, size=(T, S))
    return m

uniforms = take_uniform_draws()
#uniforms

In [109]:
# draw e_t samples from given param_guess
def draw_et(uniforms, param_vec, beta = 0.99, S = 1000, T = 100):
    #epsilon is normally distributed with mean 0, var = sigma^2
    alpha, rho, mu, sigma = param_vec
    eps_mat = norm.ppf(uniforms, 0, sigma)
    #return a matrix of epsilon draws
    return eps_mat


param_vec = (0.8,0.3,6,0.4)
et_draws = draw_et(uniforms, param_vec)
#et_draws

In [110]:
def get_zt_draws(et_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    #T = 100, ith row is the eps in t = i
    #z_0 = mu
    z_draws = np.zeros((T, S))
    z_draws[0,:] = mu
    for i in range(1, T):
        z_draws[i, :] = rho * z_draws[i-1, :] + (1-rho)*mu + et_draws[i-1, :]
    #z_draws = z_draws[1:, :]
    return z_draws

z_draws = get_zt_draws(et_draws, param_vec)
#z_draws

In [111]:
def get_kt_draws(zt_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    k_draws = np.zeros((T, S))
    #set k1 to the first observation
    k_draws[0, :] = k_vec[0]
    for i in range(1, T):
        k_draws[i, :] = alpha * beta * np.exp(zt_draws[i-1, :]) * (k_draws[i-1, :] ** alpha)
    return k_draws

k_draws = get_kt_draws(z_draws, param_vec)
#k_draws


In [112]:
def get_wt_draws(zt_draws, kt_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    
    w_draws = np.zeros((T, S))
    for i in range(T):
        w_draws[i, :] = (1-alpha)*np.exp(zt_draws[i,:]) * (kt_draws[i,:]**alpha)
        
    w_prime = (1-alpha) * np.exp(zt_draws) * (kt_draws ** alpha)
    #print((1-alpha)*np.exp(zt_draws[0,0])*kt_draws[0,0]**alpha)
    return w_draws
    
def get_rt_draws(zt_draws, kt_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    
    r_draws = np.zeros((T,S))
    
  
    
    for i in range(T):
        r_draws[i, :] = alpha * np.exp(zt_draws[i,:]) * (kt_draws[i,:]**(alpha-1))
    return r_draws


r_draws = get_rt_draws(z_draws, k_draws, param_vec)
w_draws = get_wt_draws(z_draws, k_draws, param_vec)
#r_draws

In [113]:
#c_t = w_t + r_t k_t - k_t+1
def get_ct_draws(wt_draws, rt_draws, kt_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    
    #since we don't have k_T+1, simply assume that k_T+1= k_T
    ktp1 = kt_draws[-1, :]
    k_temp = np.vstack((kt_draws, ktp1))
    #print(k_temp)
    c_draws = np.zeros((T,S))
    for i in range(T):
        c_draws[i,:] = wt_draws[i,:] + rt_draws[i,:] * k_temp[i,:] - k_temp[i+1, :]
    return c_draws

#get_ct_draws(w_draws, r_draws, k_draws, param_vec)


In [114]:
def get_yt_draws(zt_draws, kt_draws, param_vec, beta = 0.99, S = 1000, T = 100):
    alpha, rho, mu, sigma = param_vec
    
    #y_draws = np.zeros((T,S))
    y_draws = np.exp(zt_draws) * (kt_draws**alpha)
    
    return y_draws

y_draws = get_yt_draws(z_draws, k_draws, param_vec)
#y_draws

In [115]:

#first we need data moments

data_mean_c = c_vec.mean()
data_mean_k = k_vec.mean()
data_mean_cy = (c_vec/y_vec).mean()
data_var_y = y_vec.var()
data_corr_c = np.corrcoef(c_vec[:-1], c_vec[1:])[0, 1]
data_corr_ck = np.corrcoef(c_vec, k_vec)[0, 1]


In [116]:
def get_sim_model_moments(param_vec, uniforms, S = 1000):
    et_draws = draw_et(uniforms, param_vec)
    z_draws = get_zt_draws(et_draws, param_vec)
    k_draws = get_kt_draws(z_draws, param_vec)
    r_draws = get_rt_draws(z_draws, k_draws, param_vec)
    w_draws = get_rt_draws(z_draws, k_draws, param_vec)
    c_draws = get_ct_draws(w_draws, r_draws, k_draws, param_vec)
    y_draws = get_yt_draws(z_draws, k_draws, param_vec)
    
    #print(c_draws.mean(axis=0))
    sim_mean_c = c_draws.mean(axis=0).mean()
    
    sim_mean_l = k_draws.mean(axis=0).mean()
    
    sim_mean_cy = (c_draws / y_draws).mean(axis=0).mean()
    
    sim_var_y = y_draws.var(axis=0).mean()
    
    temp_c = []
    for i in range(S):
        temp_c.append(np.corrcoef(c_draws[:-1, i], c_draws[1:, i])[0, 1])
    
    sim_corr_c = np.array(temp_c).mean()
    
    temp_ck = []
    for i in range(S):
        temp_ck.append(np.corrcoef(c_draws[:,i], k_draws[:, i])[0,1])
        
    sim_corr_ck = np.array(temp_ck).mean()
    
    return np.array([sim_mean_c, sim_mean_l, sim_mean_cy, sim_var_y, sim_corr_c, sim_corr_ck])

get_sim_model_moments(param_vec, uniforms)
    


array([4.29570593e+10, 4.15794901e+12, 7.24531757e-03, 3.67965881e+25,
       3.76623812e-01, 3.78659556e-01])

In [123]:
def get_err_vec(param_vec, uniforms):
    data_moments = np.array([data_mean_c, data_mean_k, data_mean_cy, data_var_y, data_corr_c, data_corr_ck])
    sim_model_moments = get_sim_model_moments(param_vec, uniforms)
    
    return (sim_model_moments - data_moments) / data_moments

def crit(param_vec, args):
    uniforms, W = args
    err_vec = get_err_vec(param_vec, uniforms)
    return err_vec.T @ W @ err_vec

argum = [uniforms, np.eye(6)]
param_vec = (0.9,0.6,10,0.8)
crit(param_vec, argum)

soln = opt.minimize(crit, param_vec, args = argum, bounds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1)), 
                    method = 'L-BFGS-B', options={'eps': 1e-3})
soln


/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in multiply
  import sys
/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in subtract
  # This is added back by InteractiveShellApp.init_path()
/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in multiply
  """
/Users/Zeshun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kerne

      fun: 9.325769890607724
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-5.06277784e+01, -4.02757365e+00, -2.23435496e-02,  4.00493393e+01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 205
      nit: 6
   status: 0
  success: True
        x: array([ 0.01, -0.99,  5.  ,  0.01])